## Growth rates and fluxes

One of the major applications of micom is to identify growth rates and fluxes for a given community. We will return to our *E. coli* toy example.

In [1]:
from micom import Community, data

tax = data.test_taxonomy()
com = Community(tax)

### FBA

If one is only interested in the community growth rate $\mu_c$ we can use normal FBA to optimize this growth rate. By default micom assigns the community growth rate as the objective for a community model. 

In [2]:
print(com.objective.expression)
com.optimize()

1.0*community_objective


<CommunitySolution 0.874 at 0x7f1b450c2c40>

`optimize` returns a full-fledged solution object and you can inspect several aspects of it. For instance to get individual growth rates and information for the taxa:

In [3]:
sol = com.optimize()
sol.members

abundance  growth_rate  reactions  metabolites
compartments                                                      
Escherichia_coli_1       0.25     0.507620         95           72
Escherichia_coli_2       0.25     1.079031         95           72
Escherichia_coli_3       0.25     0.000000         95           72
Escherichia_coli_4       0.25     1.909035         95           72
medium                    NaN          NaN         20           20

By default micom does not return fluxes since that can be slow for realistic large community models. If you want fluxes as well you can do so by passing the `fluxes` argument to `optimize`. You can also specifiy if you would like the fluxes to be obtained by parsimonious FBA.

In [4]:
sol = com.optimize(fluxes=True, pfba=True)
sol.fluxes

reaction            ACALD  ACALDt  ACKr     ACONTa     ACONTb  ACt2r  ADK1  \
compartment                                                                  
Escherichia_coli_1    0.0     0.0   0.0   3.958207   3.958207    0.0   0.0   
Escherichia_coli_2    0.0     0.0   0.0   6.412778   6.412778    0.0   0.0   
Escherichia_coli_3    0.0     0.0   0.0   1.859626   1.859626    0.0   0.0   
Escherichia_coli_4    0.0     0.0   0.0  11.797502  11.797502    0.0   0.0   
medium                NaN     NaN   NaN        NaN        NaN    NaN   NaN   

reaction               AKGDH  AKGt2r  ALCD2x  ...       RPI  SUCCt2_2  SUCCt3  \
compartment                                   ...                               
Escherichia_coli_1  3.410537     0.0     0.0  ... -1.247066       0.0     0.0   
Escherichia_coli_2  7.103137     0.0     0.0  ... -2.675271       0.0     0.0   
Escherichia_coli_3  0.005102     0.0     0.0  ...  0.000000       0.0     0.0   
Escherichia_coli_4  9.737846     0.0     0.0  ... -5.204648       0.0     0.0   
medium                   NaN     NaN     NaN  ...       NaN       NaN     NaN   

reaction               SUCDi    SUCOAS      TALA  THD2      TKT1      TKT2  \
compartment                                                                  
Escherichia_coli_1  3.410537 -3.410537  0.791377   0.0  0.791377  0.608127   
Escherichia_coli_2  7.103137 -7.103137  1.706628   0.0  1.706628  1.317098   
Escherichia_coli_3  0.005102 -0.005102  0.000000   0.0  0.000000  0.000000   
Escherichia_coli_4  9.737846 -9.737846  3.490910   0.0  3.490910  2.801749   
medium                   NaN       NaN       NaN   NaN       NaN       NaN   

reaction                  TPI  
compartment                    
Escherichia_coli_1   4.616765  
Escherichia_coli_2   7.873710  
Escherichia_coli_3   1.857075  
Escherichia_coli_4  15.561006  
medium                    NaN  

[5 rows x 115 columns]

By defaults fluxes are stratified by taxa and the external medium to aid further analysis. 

Returning to the growth rates we can see the major problem with regular FBA for community levels. Usually it will maximize growth for only a few taxa in the model yielding unrealistically high growth rates. However, in our setup we have 5 identical *E. coli* strains and we would expect all of the taxa to grow with the same rate (see Methods section).

### Cooperative tradeoff

Cooperative tradeoff is a two step methods that allows you to get a unique solution for the the individuals growth rates that favors individual growth but still allows for a sup-optimal community growth rate. For that we always have to decide on a fraction of the maximum community growth rate we want to enforce. Our own results based on a data set of 189 gut microbiomes suggest that this tradeoff has tobe substantially lower than the optimal community growth rate to yield realistic growth rates (<=50% optimum). For our *E. coli* model we can start with getting the best solution while still maintaining optimal community growth (100% of maximum).

**Cooperative tradeoff requires a QP-capable solver such as cplex or gurobi (both have academic licenses available)!**

In [5]:
sol = com.cooperative_tradeoff(fraction=1.0)
sol

<CommunitySolution 0.874 at 0x7f1afc37a7c0>

As we see all taxa now grow at the same rate as would be expected. If we want fluxes as well we can again request those using the `fluxes` and `pfba` arguments.

In [6]:
sol = com.cooperative_tradeoff(fluxes=True, pfba=True)
sol.fluxes

reaction                   ACALD        ACALDt          ACKr   ACONTa  \
compartment                                                             
Escherichia_coli_1  2.884724e-12  1.357458e-12 -1.418608e-10  6.00725   
Escherichia_coli_2  2.884724e-12  1.357458e-12 -1.418608e-10  6.00725   
Escherichia_coli_3  2.884724e-12  1.357458e-12 -1.418608e-10  6.00725   
Escherichia_coli_4  2.884724e-12  1.357458e-12 -1.418608e-10  6.00725   
medium                       NaN           NaN           NaN      NaN   

reaction             ACONTb         ACt2r          ADK1     AKGDH  \
compartment                                                         
Escherichia_coli_1  6.00725 -1.418608e-10  3.277165e-10  5.064376   
Escherichia_coli_2  6.00725 -1.418608e-10  2.916750e-10  5.064376   
Escherichia_coli_3  6.00725 -1.418608e-10 -8.500822e-10  5.064376   
Escherichia_coli_4  6.00725 -1.418608e-10  1.388080e-10  5.064376   
medium                  NaN           NaN           NaN       NaN   

reaction                  AKGt2r        ALCD2x  ...       RPI      SUCCt2_2  \
compartment                                     ...                           
Escherichia_coli_1  3.095454e-12  1.527266e-12  ... -2.281503  2.891365e-11   
Escherichia_coli_2  3.095454e-12  1.527266e-12  ... -2.281503  2.891365e-11   
Escherichia_coli_3  3.095454e-12  1.527266e-12  ... -2.281503  2.891365e-11   
Escherichia_coli_4  3.095454e-12  1.527266e-12  ... -2.281503  2.891365e-11   
medium                       NaN           NaN  ...       NaN           NaN   

reaction                  SUCCt3     SUCDi    SUCOAS      TALA          THD2  \
compartment                                                                    
Escherichia_coli_1  2.634339e-11  5.064376 -5.064376  1.496984  2.891446e-11   
Escherichia_coli_2  2.634339e-11  5.064376 -5.064376  1.496984  2.891446e-11   
Escherichia_coli_3  2.634339e-11  5.064376 -5.064376  1.496984  2.891446e-11   
Escherichia_coli_4  2.634339e-11  5.064376 -5.064376  1.496984  2.891446e-11   
medium                       NaN       NaN       NaN       NaN           NaN   

reaction                TKT1      TKT2       TPI  
compartment                                       
Escherichia_coli_1  1.496984  1.181498  7.477382  
Escherichia_coli_2  1.496984  1.181498  7.477382  
Escherichia_coli_3  1.496984  1.181498  7.477382  
Escherichia_coli_4  1.496984  1.181498  7.477382  
medium                   NaN       NaN       NaN  

[5 rows x 115 columns]

Also if you have prior information about the growth rates you can also enforce a minimum individual growth rate for the taxa.

In [8]:
sol1 = com.cooperative_tradeoff(min_growth=0.1)  # single value
sol2 = com.cooperative_tradeoff(min_growth=[0.1, 0.2, 0.3, 0.4])  # one value for each taxa
print(sol1, sol2)

<CommunitySolution 0.874 at 0x7f1afc37aa90> <CommunitySolution 0.874 at 0x7f1afc36c1f0>


Finally, you might want to see the impact of the tradeoff parameter on your solution. For this you can simply pass in an array-like type as the `fraction` parameter.

In [9]:
import numpy as np

sols = com.cooperative_tradeoff(fraction=np.arange(0.1, 1.01, 0.1))
sols

tradeoff                                     solution
0       1.0  <CommunitySolution 0.874 at 0x7f1b450c27f0>
1       0.9  <CommunitySolution 0.787 at 0x7f1afc4adb20>
2       0.8  <CommunitySolution 0.699 at 0x7f1afc3d0fa0>
3       0.7  <CommunitySolution 0.612 at 0x7f1afc36c040>
4       0.6  <CommunitySolution 0.524 at 0x7f1afc36c0a0>
5       0.5  <CommunitySolution 0.437 at 0x7f1afc36cd00>
6       0.4  <CommunitySolution 0.350 at 0x7f1afc36cd30>
7       0.3  <CommunitySolution 0.262 at 0x7f1afc410220>
8       0.2  <CommunitySolution 0.175 at 0x7f1afc410430>
9       0.1  <CommunitySolution 0.087 at 0x7f1afc458100>

The solutions can than be inspected by the usual pandas methods. See the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/) for more infos.

In [10]:
rates = sols.solution.apply(lambda x: x.members.growth_rate)
rates

compartments  Escherichia_coli_1  Escherichia_coli_2  Escherichia_coli_3  \
0                       0.873922            0.873922            0.873922   
1                       0.786529            0.786529            0.786529   
2                       0.699137            0.699137            0.699137   
3                       0.611745            0.611745            0.611745   
4                       0.524353            0.524353            0.524353   
5                       0.436961            0.436961            0.436961   
6                       0.349569            0.349569            0.349569   
7                       0.262176            0.262176            0.262176   
8                       0.174784            0.174784            0.174784   
9                       0.087392            0.087392            0.087392   

compartments  Escherichia_coli_4  medium  
0                       0.873922     NaN  
1                       0.786529     NaN  
2                       0.699137     NaN  
3                       0.611745     NaN  
4                       0.524353     NaN  
5                       0.436961     NaN  
6                       0.349569     NaN  
7                       0.262176     NaN  
8                       0.174784     NaN  
9                       0.087392     NaN